In [20]:
import netCDF4
import numpy as np

input_ncfile_path = 'gk2a_ir096_2021/gk2a_ami_le1b_ir096_ko020lc_202101010000.nc' 
CT_path='./conversion_table/'
output_txt_path='sample_output.txt'
output_npy_path='sample_image_pixel_data.npy'
ncfile = netCDF4.Dataset('gk2a_ir096_202201\gk2a_ami_le1b_ir096_ko020lc_202201010000.nc','r',format='netcdf4')
ipixel=ncfile.variables['image_pixel_values']
ipixel_process = ipixel[:]


In [21]:
channel=ipixel.getncattr('channel_name')
if ((channel == 'VI004') or (channel == 'NR016')
    or (channel == 'VI005')):
        mask = 0b0000011111111111 #11bit mask
elif ((channel == 'VI006')
    or (channel == 'NR013') or (channel == 'WV063')):
        mask = 0b0000111111111111 #12bit mask
elif (channel == 'SW038'):
    mask = 0b0011111111111111 #14bit mask
else:
    mask = 0b0001111111111111 #13bit mask
    ipixel_process_masked=np.bitwise_and(ipixel_process,mask)

In [22]:
channel

'IR096'

In [3]:
AL_postfix='_con_alb.txt'
BT_postfix='_con_bt.txt'
if (channel[0:2] == 'VI') or (channel[0:2] == 'NR'):
    conversion_table=np.loadtxt(
    cal_table_path+channel+AL_postfix,'float64')
else:
    conversion_table=np.loadtxt(
        CT_path+channel+BT_postfix,'float64')
ipixel_process_masked_converted=conversion_table[
    ipixel_process_masked]

NameError: name 'cal_table_path' is not defined

In [23]:
ipixel_process_masked

masked_array(
  data=[[6923, 6926, 6904, ..., 6701, 6706, 6723],
        [6923, 6926, 6907, ..., 6708, 6718, 6732],
        [6928, 6931, 6930, ..., 6708, 6718, 6732],
        ...,
        [6094, 6117, 6139, ..., 5450, 5467, 5489],
        [6185, 6117, 6139, ..., 5503, 5469, 5489],
        [6185, 6189, 6195, ..., 5533, 5520, 5490]],
  mask=False,
  fill_value=999999,
  dtype=uint16)

In [24]:
print(ipixel_process)

[[6923 6926 6904 ... 6701 6706 6723]
 [6923 6926 6907 ... 6708 6718 6732]
 [6928 6931 6930 ... 6708 6718 6732]
 ...
 [6094 6117 6139 ... 5450 5467 5489]
 [6185 6117 6139 ... 5503 5469 5489]
 [6185 6189 6195 ... 5533 5520 5490]]


In [11]:
data2 = np.load('dataset/train/subset_010462_01.npy')

In [13]:
data2.shape

(40, 40, 15)

In [14]:
data2

array([[[174.67710876,  97.50983429, 203.54122925, ..., 159.64146423,
           5.57419205,   0.        ],
        [174.91165161,  96.68958282, 203.99794006, ..., 159.64146423,
           5.57419205,   0.        ],
        [173.59732056,  97.98129272, 204.01615906, ..., 159.64146423,
           5.57419205,   0.        ],
        ...,
        [173.03848267,  95.99974823, 199.57099915, ..., 161.20739746,
           4.79637623,   0.        ],
        [173.87191772,  96.29589844, 199.39268494, ..., 161.26597595,
           4.81756973,   0.        ],
        [173.28308105,  95.21756744, 199.53744507, ..., 161.30795288,
           4.79751778,   0.        ]],

       [[173.48847961,  97.38205719, 201.76010132, ..., 159.69039917,
           5.69830656,   0.        ],
        [174.80511475,  96.91531372, 201.47776794, ..., 159.69039917,
           5.69830656,   0.        ],
        [174.28555298,  97.36418152, 201.19615173, ..., 159.67408752,
           5.65690517,   0.        ],
        ...,


In [7]:
ipixel_process.shape

(1578, 1276)